# Exploratory Data Analysis and Basic Imputation of Missing Values

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.feature_selection import chi2, f_regression, f_classif, mutual_info_classif, mutual_info_regression 
from sklearn.feature_selection import SelectKBest, SelectFromModel, VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import neighbors
from sklearn import ensemble
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

from matplotlib import pyplot as plt

from ipynb.fs.full.Self_Written_Functions_Sheet_Recover import col_obj_list, nan_col_list, cols_nan_unique, nan_col_obj_list_fillna, fill_masvnrtype

# Data Exploration

In [3]:
hp_w_outcome = pd.read_csv('../train.csv')
hp_w_outcome.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
hp_saleprice = hp_w_outcome[['SalePrice']]

hp = hp_w_outcome.drop(['SalePrice'], axis=1)
hp

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Un

In [5]:
hp_saleprice

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [8]:
hp['BsmtBaths'] = hp['BsmtFullBath'] + hp['BsmtHalfBath']
hp['BsmtBaths']

0       1
1       1
2       1
3       1
4       1
       ..
1455    0
1456    1
1457    0
1458    1
1459    1
Name: BsmtBaths, Length: 1460, dtype: int64

In [ ]:
hp['TotalBaths'] = hp['FullBath'] + hp['HalfBath']

In [6]:
# hp.to_csv('../hp.csv', index=False)

In [7]:
hp_saleprice.to_csv('../hp_saleprice.csv', index=False)

In [ ]:
hp_saleprice['SalePrice'].hist(bins=range(min(hp_saleprice['SalePrice']), 
                                          max(hp_saleprice['SalePrice']) + 1, 25000),
                               figsize=(20,10))

In [ ]:
plt.scatter(y=np.log(hp_saleprice['SalePrice']), x=hp.GrLivArea)

In [ ]:
hp_logsaleprice = pd.DataFrame(np.log(hp_saleprice))
hp_logsaleprice

In [ ]:
hp_logsaleprice.hist(figsize=(20,10))

In [ ]:
hp_logsaleprice.to_csv('../hp_logsaleprice.csv', index=False)

## General info

In [ ]:
hp.info()

# [LotFrontage, Alley, MasVnrType, MasVnrArea]

In [ ]:
hp.columns

In [ ]:
hp.describe()

## Use First 2 Functions from Functions_Sheet.ipynb to Get Columns with Null Values

In [ ]:
# return a list with the names of columns that contain nans

# columns that have dtype=='object' vs not
    # to use with col_obj_list
cols_obj = []
cols_num = []

# columns that have 0 as a value
    # to use with col_num_list_0
cols_0 = []

# columns with NaN values
    # to use with list_nan_col
cols_w_nan_obj = []
cols_w_nan_num = []


## Split Column Names into dtype Object or Other

In [ ]:
# columns that have dtype=='object' vs not

cols_obj = []
cols_num = []

# Creates list of column names with dtype=='object' and one where dtype!='object'
col_obj_list(hp, hp.columns, cols_obj, cols_num)

In [ ]:
# columns that have dtype=='object'

cols_obj

In [ ]:
# columns that do NOT have dtype=='object'

cols_num

## Get Columns with Null Values

In [ ]:
# columns with object dtype that have null values

cols_w_nan_obj = []

# Outputs a list of the columns that have null values
nan_col_list(hp, cols_obj, cols_w_nan_obj)

In [ ]:
# columns with non-object dtype that have null values

cols_w_nan_num = []

# Outputs a list of the columns that have null values
nan_col_list(hp, cols_num, cols_w_nan_num)

In [ ]:
np.sum(hp[hp['Alley'].isna()]['Alley'])

In [ ]:
np.sum(hp['Alley'].isna())

# Explore Null Values of the Columns

## Find Unique Values of Columns of Object dtype Excluding NaNs

In [ ]:
data_dict_obj = {}
data_dict_null_obj = {}
cols_nan_unique(hp, cols_w_nan_obj, data_dict_obj, data_dict_null_obj)

# to visualize via dataframes the unique values that exist in the dataset
# to visualize the null count next to those values

data_dict_obj_df = pd.DataFrame.from_dict(data_dict_obj, orient='index').fillna('')
data_dict_null_obj_df = pd.DataFrame(data_dict_null_obj.items())
data_dict_null_obj_df.columns = ['Col_Name', 'Null_Count']
data_dict_null_obj_df = data_dict_null_obj_df.set_index('Col_Name')
data_dict_null_obj_df

In [ ]:
data_dict_obj_df

In [ ]:
pd.concat([data_dict_obj_df, data_dict_null_obj_df], axis=1)

In [ ]:
hp[hp['MasVnrType']=='None']

### Columns with dtype Object that have 'None' but also have NaNs


- MasnVnrType

### Will use alternative to 'No' or 'None' as those are used in the data:

- Will use 'DNE'

## Replace Appropriate Columns' NaNs with 'DNE'

### Columns with dtype Object that should NOT have NaNs:

In [ ]:
should_not_have_nan_obj = ['MasVnrType', 'Electrical']

### Columns with dtype non-Object that should NOT have NaNs:

In [ ]:
should_not_have_nan_num = ['LotFrontage', 'MasVnrArea']

In [ ]:
obj_DNE_list = [i for i in cols_w_nan_obj if i not in should_not_have_nan_obj]

In [ ]:
num_DNE_list = [i for i in cols_w_nan_num if i not in should_not_have_nan_num]

In [ ]:
nan_col_obj_list_fillna(hp, obj_DNE_list)

In [ ]:
nan_col_obj_list_fillna(hp, num_DNE_list)

### Check if There Are Any NaNs Remaining

In [ ]:
cols_w_nan_obj2 = []

nan_col_list(hp, cols_w_nan_obj, cols_w_nan_obj2)

In [ ]:
cols_w_nan_num2 = []

nan_col_list(hp, cols_w_nan_num, cols_w_nan_num2)

In [ ]:
cols_obj

In [ ]:
hp['Electrical'].unique()

In [ ]:
hp[hp['Electrical'].isna()]

In [ ]:
hp['Electrical'].mode()

In [ ]:
hp['Electrical'].fillna(hp['Electrical'].mode()[0], inplace=True)

In [ ]:
hp[hp['Electrical'].isna()]

## Find Unique Values of Columns of Non-Object dtype Excluding NaNs

In [ ]:
data_dict_num = {}
data_dict_null_num = {}
cols_nan_unique(hp, cols_w_nan_num, data_dict_num, data_dict_null_num)

# data_dict_obj_df = pd.DataFrame.from_dict(data_dict_obj, orient='index').fillna('')
data_dict_null_num_df = pd.DataFrame(data_dict_null_num.items())
data_dict_null_num_df.columns = ['Col_Name', 'Null_Count']
data_dict_null_num_df = data_dict_null_num_df.set_index('Col_Name')
data_dict_null_num_df

# Fill NaNs and like-NaNs

#### LotFrontage

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html

In [ ]:
np.sum(hp[hp['LotFrontage']==0]['LotFrontage'])

In [ ]:
plt.scatter(x=hp['LotFrontage'], y=hp['LotArea'])

#### MasVnrArea

In [ ]:
np.sum(hp[hp['MasVnrArea']==0]['MasVnrArea'])

In [ ]:
# safer to assume that there is none (0)

hp['MasVnrArea'].hist(bins=30, figsize=(20,10))

In [ ]:
np.sum(hp['MasVnrArea'].isna())

In [ ]:
hp.loc[hp.MasVnrArea.isna(), 'MasVnrArea'] = 0
hp[hp['MasVnrArea'].isna()]

In [ ]:
np.sum(hp['MasVnrArea'].isna())

### Check MasVnrArea for any NaNs

In [ ]:
np.sum(hp['MasVnrType'].isna())

#### Cross-Check MasVnrType and MasVnrArea

In [ ]:
hp[hp['MasVnrType'].isna()][['MasVnrType', 'MasVnrArea']]

In [ ]:
hp.loc[(hp.MasVnrType.isna()) & (hp.MasVnrArea==0), 'MasVnrType']='None'

In [ ]:
hp[hp['MasVnrType'].isna()][['MasVnrType', 'MasVnrArea']]

#### Check for discrepancies between Type and Area

In [ ]:
# Will use mode of MasVnrType if MasVnrArea is > 1

hp[(hp['MasVnrType']=='None') & (hp['MasVnrArea']!=0)][['MasVnrType', 'MasVnrArea']]

In [ ]:
# Will assume MasVnrArea of 1 are actually 0 based on previous cell

hp.loc[hp.MasVnrArea==1, 'MasVnrArea'] = 0

In [ ]:
hp[(hp['MasVnrType']=='None') & (hp['MasVnrArea']!=0)][['MasVnrType', 'MasVnrArea']]

In [ ]:
hp['MasVnrType'].value_counts()

In [ ]:
hp[hp['MasVnrType']=='BrkFace']['MasVnrArea'].hist(bins=40, figsize=(20,10))

In [ ]:
hp[hp['MasVnrType']=='Stone']['MasVnrArea'].hist(bins=30, figsize=(20,10))

In [ ]:
hp[hp['MasVnrType']=='BrkCmn']['MasVnrArea'].hist(bins=15, figsize=(20,10))

#### Based on the Graphs above, we can see that BrkFace has the highest count

In [ ]:
# ['MasVnrType'] should = the mode of the ['MasVnrType'] that is NOT None

from collections import Counter

masvnrtype_mode = [x[0] for x in Counter(list(hp['MasVnrType'])).most_common()]
    # outputs something that looks like: ['None', 'BrkFace', 'Stone', 'BrkCmn']

if masvnrtype_mode[0]=='None':
    hp.loc[(hp.MasVnrArea > 1) & (hp.MasVnrType == 'None'), 'MasVnrType'] = masvnrtype_mode[1]
else:
    hp.loc[(hp.MasVnrArea > 1) & (hp.MasVnrType == 'None'), 'MasVnrType'] = masvnrtype_mode[0]

In [ ]:
hp[(hp['MasVnrType']=='None') & (hp['MasVnrArea']!=0)][['MasVnrType', 'MasVnrArea']]

# we have successfully converted MasVnrType of 'None' to the most common VnrType that is NOT 'None' as long as the
# MasVnrArea is greater than 0

In [ ]:
hp.loc[624, ['MasVnrType', 'MasVnrArea']]

In [ ]:
hp.loc[1300, ['MasVnrType', 'MasVnrArea']]

In [ ]:
hp.loc[1334, ['MasVnrType', 'MasVnrArea']]

In [ ]:
hp['MasVnrType'].unique()

In [ ]:
# Here we see that there are 2 times when MasVnrArea is 0, there is a Type recorded

# maybe use mean or mode of MasVnrArea for each of the MasVnrType?
    # Maybe use KNN?


hp[(hp['MasVnrType']!='None') & (hp['MasVnrArea']==0)][['MasVnrType', 'MasVnrArea']]

In [ ]:
print(hp.loc[hp['MasVnrType']=='BrkFace', 'MasVnrArea'].mean())
print(hp.loc[hp['MasVnrType']=='BrkFace', 'MasVnrArea'].median())
print(hp.loc[hp['MasVnrType']=='BrkFace', 'MasVnrArea'].mode())

#### Replace values in MasVnrArea that have a Type, but are == 0

- use Mode

In [ ]:
list(hp[(hp['MasVnrType']!='None') & (hp['MasVnrArea']==0)]['MasVnrType'])

In [ ]:
mvt_list = set(list(hp[(hp['MasVnrType']!='None') & (hp['MasVnrArea']==0)]['MasVnrType']))

fill_masvnrtype(hp, mvt_list)

In [ ]:
print(hp.loc[hp['MasVnrType']=='BrkFace', 'MasVnrArea'].mean())
print(hp.loc[hp['MasVnrType']=='BrkFace', 'MasVnrArea'].median())
print(hp.loc[hp['MasVnrType']=='BrkFace', 'MasVnrArea'].mode())

In [ ]:
print(hp.loc[hp['MasVnrType']=='Stone', 'MasVnrArea'].mean())
print(hp.loc[hp['MasVnrType']=='Stone', 'MasVnrArea'].median())
print(hp.loc[hp['MasVnrType']=='Stone', 'MasVnrArea'].mode())

In [ ]:
hp[(hp['MasVnrType'].isna()) & (hp['MasVnrArea']!=0)][['MasVnrType', 'MasVnrArea']]

In [ ]:
hp[(hp['MasVnrType']!='None') & (hp['MasVnrArea'].isna())][['MasVnrType', 'MasVnrArea']]

In [ ]:
np.sum(hp[hp['GarageYrBlt']==0]['GarageYrBlt'])

## Replace null in GarageYrBlt and change to 'YrSold'


In [ ]:
hp[hp['GarageYrBlt']==0]

In [ ]:
hp[hp['YearBuilt'].isna()]

In [ ]:
hp[hp['YearRemodAdd'].isna()]

In [ ]:
hp[hp.YearBuilt>hp.YearRemodAdd]

In [ ]:
hp[hp.YearBuilt<hp.YearRemodAdd]

In [ ]:
hp[hp.YearBuilt==hp.YearRemodAdd]

In [ ]:
hp[hp['GarageYrBlt']==0]

In [ ]:
hp.loc[hp.GarageYrBlt==0]

In [ ]:
# adds the column 'GarageYrModified' and sets it == to 'YearRemodAdd' or 'GarageYrBlt'

# https://www.redfin.com/news/does-a-garage-add-value-to-a-home/

# according to the above, having a garage does not seem to impact houses in the midwest (besides Chicago)

hp['GarageYrModified'] = np.where(hp['YearRemodAdd']>hp['YearBuilt'], hp['YearRemodAdd'], hp['GarageYrBlt'])

hp['GarageYrModified'] = np.where(hp['GarageYrModified']==0, hp['YearRemodAdd'], hp['YearBuilt'])
hp['GarageYrModified'] = pd.to_numeric(hp['GarageYrModified'])

In [ ]:
hp[hp['GarageYrModified']==0]

In [ ]:
hp[hp['GarageYrModified'].isna()]

## Replace Appropriate Columns' NaNs with 'DNE'

In [ ]:
obj_DNE_list = [i for i in cols_w_nan_obj if i not in should_not_have_nan_obj]

In [ ]:
num_DNE_list = [i for i in cols_w_nan_num if i not in should_not_have_nan_num]

In [ ]:
nan_col_obj_list_fillna(hp, obj_DNE_list)

In [ ]:
nan_col_obj_list_fillna(hp, num_DNE_list)

### Check if There Are Any NaNs Remaining

In [ ]:
cols_w_nan_obj2 = []

nan_col_list(hp, cols_w_nan_obj, cols_w_nan_obj2)

In [ ]:
cols_w_nan_num2 = []

nan_col_list(hp, cols_w_nan_num, cols_w_nan_num2)

In [ ]:
hp.to_csv('../hp_for_dummies.csv', index=False)